<a href="https://colab.research.google.com/github/ricardoskewes/68610finalprojectgroup48/blob/colab_branch/truly_final_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%bash
!(stat -t /usr/local/lib/*/dist-packages/google/colab > /dev/null 2>&1) && exit
rm -rf 68610finalprojectgroup48
git clone https://github.com/ricardoskewes/68610finalprojectgroup48.git


Cloning into '68610finalprojectgroup48'...
Updating files: 100% (5007/5007), done.


In [2]:
!pip install sacrebleu simpletransformers
!pip install wordfreq
!pip install wandb


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.3/316.3 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 48.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.7 MB/s eta 0:00

# Setup and imports

In [3]:

# Append the cloned repository to sys.path

import sys
sys.path.append("/content/68610finalprojectgroup48")
projectDir = "/content/68610finalprojectgroup48/tweepfake"
sys.path.insert(0, projectDir)
resultsDir = projectDir+"/data/results"

# Standard libraries
import os
import csv
import logging
import math
import datetime

# Data manipulation libraries
import numpy as np
import pandas as pd

# NLP libraries
import spacy
from textblob import TextBlob
from wordfreq import word_frequency

#  confusion matrix stuff
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Machine learning libraries
import torch
from transformers import set_seed
from simpletransformers.classification import ClassificationModel, ClassificationArgs

# wandb library
import wandb

#DataHandler
from DataHandler import DataHandler


# Set random seed for reproducibility
random_state = 523
set_seed(random_state)


# Data loading

In [4]:
# Data loading code (unchanged)
from DataHandler import DataHandler
dh = DataHandler()

# Paths to datasets
csvTrainDataset = projectDir + "/data/splits/train.csv"
csvValDataset = projectDir + "/data/splits/validation.csv"
csvTestDataset = projectDir + "/data/splits/test.csv"

# Load datasets
dfTrain = dh.readCSVData(csvTrainDataset)
dfVal = dh.readCSVData(csvValDataset)
dfTest = dh.readCSVData(csvTestDataset)

# Select interesting columns for this study
dfTrainDataset = dfTrain[["screen_name", "text", "account.type"]]
dfValDataset = dfVal[["screen_name", "text", "account.type"]]
dfTestDataset = dfTest[["screen_name", "text", "account.type"]]

# Prepare training data
X_train_all = dfTrainDataset.drop(columns=['screen_name'])
X_train_all.columns = ["text", "label"]

X_val_all = dfValDataset.drop(columns=['screen_name'])
X_val_all.columns = ["text", "label"]

X_test_all = dfTestDataset.drop(columns=['screen_name'])
X_test_all.columns = ["text", "label"]

# Map labels to integers
dictLabels = {"human": 0, "bot": 1}

X_train_all["label"] = X_train_all["label"].apply(lambda x: dictLabels[x])
X_val_all["label"] = X_val_all["label"].apply(lambda x: dictLabels[x])
X_test_all["label"] = X_test_all["label"].apply(lambda x: dictLabels[x])

# Extract labels
y_train = X_train_all["label"]
y_val = X_val_all["label"]
y_test = X_test_all["label"]

train_labels = y_train.tolist()
val_labels = y_val.tolist()
test_labels = y_test.tolist()


# Feature Functions

In [5]:
# Feature creating functions (unchanged)
spacy_nlp = spacy.load("en_core_web_sm")

def get_spacy_doc(text):
    return spacy_nlp(text)

def switch_spacy_to_text(func):
    def inner1(*args, **kwargs):
        spacy_doc = get_spacy_doc(args[0])
        return func(spacy_doc, **kwargs)
    return inner1

def count_words(text):
    return len(text.split())  # C1

def avg_word_length(text):
    words = text.split()  # C2
    return np.mean([len(w) for w in words])

@switch_spacy_to_text
def get_ANP(spacy_doc):
    # Get total length
    total_tokens = len([token for token in spacy_doc if not token.is_punct and not token.is_space])
    # List of adjectives, nouns, and pronouns ('ANP')
    adjectives = [token.text for token in spacy_doc if token.pos_ == "ADJ"]
    nouns = [token.text for token in spacy_doc if token.pos_ == "NOUN"]
    pronouns = [token.text for token in spacy_doc if token.pos_ == "PRON"]
    # Estimate densities
    adj_density = len(adjectives) / total_tokens
    noun_density = len(nouns) / total_tokens
    pronoun_density = len(pronouns) / total_tokens
    return adjectives, nouns, pronouns, adj_density, noun_density, pronoun_density

def get_ANP_clean(text):
    _, _, _, adj_density, noun_density, pronoun_density = get_ANP(text)
    return adj_density, noun_density, pronoun_density  # C3

@switch_spacy_to_text
def get_capitalizations(spacy_doc):
    capitalizations = [token.text for token in spacy_doc if token.text.isupper()]
    return capitalizations, len(capitalizations) / len(spacy_doc)

def get_capitalizations_clean(text):
    _, cap_ratio = get_capitalizations(text)
    return np.round(cap_ratio, 2)  # C4

@switch_spacy_to_text
def get_sentiment(spacy_doc):
    polarity, subjectivity = TextBlob(spacy_doc.text).sentiment
    return np.round(polarity, 3), np.round(subjectivity, 3)  # C5

@switch_spacy_to_text
def calculate_rarity_scores(spacy_doc, lang='en'):
    adj_rarity_scores = []
    noun_rarity_scores = []

    for token in spacy_doc:
        if token.pos_ == "ADJ":
            adjective = token.text.lower()
            freq = word_frequency(adjective, lang)
            adj_rarity_score = -math.log(freq) if freq > 0 else 0
            adj_rarity_scores.append(adj_rarity_score)
        if token.pos_ == "NOUN":
            noun = token.text.lower()
            freq = word_frequency(noun, lang)
            noun_rarity_score = -math.log(freq) if freq > 0 else 0
            noun_rarity_scores.append(noun_rarity_score)
    res = [0, 0]
    # Calculate median rarity score over all NAs
    if noun_rarity_scores:
        res[0] = np.median(noun_rarity_scores)
    if adj_rarity_scores:
        res[1] = np.median(adj_rarity_scores)
    return tuple(np.round(res, 3))

@switch_spacy_to_text
def get_punctuation(spacy_doc):
    punctuation = [token.text for token in spacy_doc if token.pos_ == "PUNCT"]
    return punctuation, len(punctuation) / len(spacy_doc)

def get_punctuation_clean(text):
    _, punct_ratio = get_punctuation(text)
    return np.round(punct_ratio, 2)  # C7

# Dictionary of classical features
C = {
    1: count_words,             # C1: Number of words in sentence
    2: avg_word_length,         # C2: Average word length in sentence
    3: get_ANP_clean,           # C3: Density scores of adjectives, pronouns, and nouns
    4: get_capitalizations_clean,  # C4: Density of capital letters
    5: get_sentiment,           # C5: Sentiment analysis
    6: calculate_rarity_scores, # C6: Noun and adjective rarity scores
    7: get_punctuation_clean    # C7: Density of punctuation
}


# Model definitions

In [33]:
# Model descriptions
model_descriptions = {
    1: 'BERT*(tweet)',
    2: 'BERT*({tweet <SEP> C1 <SEP> C2 ...})',
    3: 'BERT*({tweet <FEAT> C1 <FEAT> C2 ...})',
}

# Model-specific arguments (excluding hyperparameters). THESE ARE NOT MEANT TO BE SWEPT OVER, BUT JUST DEFINED.
model_args_dict = {
    1: {'transformer_type': 'bert', 'transformer_name': 'bert-base-cased', 'output-dir': 'type1'},
    2: {'transformer_type': 'bert', 'transformer_name': 'bert-base-cased', 'output-dir': 'type2'},
    3: {'separator_token_name': 'FEAT', 'transformer_type': 'bert', 'transformer_name': 'bert-base-cased', 'output-dir': 'type3'},  # Additional arguments can be added here
    # Add arguments for models 4 to 9
}

# Define which hyperparameters are applicable to each model. THESE ARE MEANT TO BE ABLE TO BE SWEPT OVER. OVERRIDE DEFAULTS USING WANDB SWEEPS
model_hyperparameters = {
    1: {"learning_rate": 1e-5, "train_batch_size": 16, "num_train_epochs": 3, "max_seq_length": 128},
    2: {"learning_rate": 1e-5, "train_batch_size": 16, "num_train_epochs": 3, "max_seq_length": 128, "C_ids": [1,2]},
    3: {"learning_rate": 1e-5, "train_batch_size": 16, "num_train_epochs": 3, "max_seq_length": 128, "C_ids": [3,4,5]},
}

#  Initialize wandb


In [34]:
# Initialize wandb
import wandb
wandb.login()

# Set the wandb project name
WANDB_PROJECT_NAME = "6-861-finalproj"


# Prepare data

In [ ]:



# all this thing outputs is the dictionary needed to append special tokens
def create_special_tokens_dict(model_id, model_args_dict):
    """
    Create a dictionary of special tokens based on the model_id.

    Args:
        model_id (int): The ID of the model to determine special tokens.
        model_args_dict (dict): Dictionary of model-specific arguments.
        hyperparameters (dict): Dictionary of hyperparameters for training.
    Returns:
        dict: Dictionary of special tokens.
    """
    additional_tokens = []
    if model_id == 3:
        additional_tokens.append(model_args_dict['separator_token_name'])
    return {'additional_special_tokens': additional_tokens}



# This thing takes in a
def initialize_model_with_special_tokens (model_id, model_args_dict, hyperparameters, special_tokens_dict, model_descriptions):
    """
    Initialize the model and tokenizer with special tokens.

    Parameters:
        transformer_type (str): The type of the transformer (e.g., 'bert').
        transformer_name (str): The name of the pretrained transformer model.
        model_id (int): The ID of the model.
        hyperparameters (dict): Dictionary of hyperparameters, both for training (i.e., learning_rate, max_seq_length, etc.) and otherwise fine-tunable (for some model_ids an example is numbeer of classical features).
        model_args_dict: (dict): Dictionary of model-specific arguments.
        special_tokens_dict (dict): Dictionary of special tokens to add.

    Returns:
        model: The initialized model.
        tokenizer: The tokenizer with added special tokens.
    """

    # Initialize model arguments
    model_args = ClassificationArgs()
    # Update model_args with hyperparameters applicable to simpletransformers... commenting out since I'm not sure if should instead pass in everything...
    # simpletransformers_params = ['learning_rate', 'train_batch_size', 'num_train_epochs', 'max_seq_length']
    # for param in simpletransformers_params:
    #     if param in hyperparameters:
    #         setattr(model_args, param, hyperparameters[param])
    for param in hyperparameters:
          setattr(model_args, param, hyperparameters[param])
    for param in model_args_dict[model_id]:
        setattr(model_args, param, model_args_dict[model_id][param])
    for param in model_descriptions[model_id]:
        setattr(model_args, param, model_descriptions[model_id][param])

    model_args.manual_seed = random_state
    model_args.overwrite_output_dir = True
    model_args.reprocess_input_data = True
    model_args.output_dir = f"outputs/model_{model_id}"

    model = ClassificationModel(model_args_dict['transformer_type'], model_args_dict['transformer_name'], args=model_args, use_cuda=torch.cuda.is_available())
    tokenizer = model.tokenizer

    # Add new special tokens
    if special_tokens_dict:
        tokenizer.add_special_tokens(special_tokens_dict)
        model.model.resize_token_embeddings(len(tokenizer))

    return model, tokenizer


def encode_with_tokens(
      tweet,
      tokenizer,
      feat_sep_token_id=None,
      C_functions=None,
      initial_token_id=None,
      max_feature_tokens=None,
      max_text_tokens=None
  ):
      """
      Encode a text and additional features with separation tokens, providing more flexible
      control over how much space features and text occupy.

      Parameters:
          tweet (str): The main text to encode.
          tokenizer: The Hugging Face tokenizer to use for encoding.
          feat_sep_token_id (int): Token ID for the feature separator token. If None,
              defaults to the model's sep or eos token.
          C_functions (list): List of feature functions that take the tweet as input and
              return a feature value (string, number, or tuple).
          initial_token_id (int): The initial token ID (e.g., CLS or BOS). If None,
              defaults to tokenizer.cls_token_id or tokenizer.bos_token_id.
          max_feature_tokens (int): Maximum total number of tokens allocated to features.
              If None, no specific limit other than model_max_length is enforced.
          max_text_tokens (int): Maximum number of tokens allocated to the main text.
              If None, no specific limit other than model_max_length is enforced.

      Returns:
          list: The final encoded sequence of token IDs.
      """
      # Determine initial token IDs and defaults
      if initial_token_id is None:
          initial_token_id = tokenizer.cls_token_id or tokenizer.bos_token_id

      if feat_sep_token_id is None:
          feat_sep_token_id = tokenizer.sep_token_id or tokenizer.eos_token_id

      model_max_length = tokenizer.model_max_length

      # Start sequence with initial token
      encoded_sequence = [initial_token_id]

      # Encode features
      feature_tokens = []
      if C_functions is not None:
          for func in C_functions:
              feature_tokens.append(feat_sep_token_id)
              feature_value = func(tweet)
              # Convert tuple values to space-separated string
              if isinstance(feature_value, tuple):
                  feature_value = ' '.join(map(str, feature_value))
              else:
                  feature_value = str(feature_value)

              encoded_feature = tokenizer.encode(feature_value, add_special_tokens=False)
              feature_tokens.extend(encoded_feature)

          # If there's a max_feature_tokens limit, truncate
          if max_feature_tokens is not None and len(feature_tokens) > max_feature_tokens:
              feature_tokens = feature_tokens[:max_feature_tokens]

      # Add the features to the main sequence (so far just initial token + features)
      encoded_sequence.extend(feature_tokens)

      # Encode the main tweet text
      encoded_tweet = tokenizer.encode(tweet, add_special_tokens=True)

      # If the initial token of encoded_tweet matches our initial_token_id (e.g., CLS), remove it
      if encoded_tweet and encoded_tweet[0] == initial_token_id:
          encoded_tweet = encoded_tweet[1:]

      # Check how many tokens are still available before hitting model_max_length
      # Reserve one token for the final separator
      available_for_text = model_max_length - len(encoded_sequence) - 1
      if max_text_tokens is not None:
          # Further restrict available tokens for text if a max_text_tokens limit is set
          available_for_text = min(available_for_text, max_text_tokens)

      # Truncate text if necessary
      if len(encoded_tweet) > available_for_text:
          encoded_tweet = encoded_tweet[:available_for_text]

      # Add the truncated tweet tokens
      encoded_sequence.extend(encoded_tweet)

      # Add final separator token at the end
      encoded_sequence.append(feat_sep_token_id)

      # In case we somehow exceed model_max_length after all operations, truncate
      if len(encoded_sequence) > model_max_length:
          encoded_sequence = encoded_sequence[:model_max_length]

      return encoded_sequence


def encode_with_tokens_based_on_model_id(tweet, tokenizer, model_id, model_args_dict, model_hyperparameters):
  if 'separator_token_name' in model_args_dict[model_id]:
    feat_sep_token_id = tokenizer.convert_tokens_to_ids(model_args_dict['separator_token_name'])
  else:
    feat_sep_token_id = None

  if 'C_ids' in model_hyperparameters[model_id]:
    C_functions = [C[i] for i in model_hyperparameters['C_ids']]
  elif 'C_ids' in model_args_dict[model_id]:
    C_functions = [C[i] for i in model_args_dict['C_ids']]
  else:
    C_functions = None

  if 'max_feature_tokens' in model_hyperparameters[model_id]:
    max_feature_tokens = model_hyperparameters[model_id]['max_feature_tokens']
  elif 'max_feature_tokens' in model_args_dict[model_id]:
    max_feature_tokens = model_args_dict[model_id]['max_feature_tokens']
  else:
    max_feature_tokens = None

  if 'max_seq_length' in model_hyperparameters[model_id]:
    max_seq_length = model_hyperparameters[model_id]['max_seq_length']
  elif 'max_seq_length' in model_args_dict[model_id]:
    max_seq_length = model_args_dict[model_id]['max_seq_length']
  else:
    max_seq_length = None

  return encode_with_tokens(tweet,
                            tokenizer,
                            feat_sep_token_id = feat_sep_token_id,
                            C_functions= C_functions,
                            initial_token_id = None,
                            max_feature_tokens=max_feature_tokens,
                            max_seq_length =  max_seq_length)



# Training loop

In [21]:

def train_model(X_train, X_val, model_id, model_hyperparameters, model_args_dict, wandb_project_name):
    """
    Train a model based on the model_id and hyperparameters.

    Args:
        model_id (int): The ID of the model to train.
        hyperparameters (dict): Dictionary of hyperparameters for training.

    Returns:
        model: The trained model.
    """

    # Set up wandb run
    current_time = datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
    run_name = f"Model_{model_id}_{current_time}"
    tags = [
        f"model_{model_id}",
        f"date_{current_time.split('_')[0]}",
    ]
    config_stuff = {**model_args_dict[model_id], **model_hyperparameters[model_id]}
    # Add hyperparameters to tags
    for param in config_stuff:
        tags.append(f"{param}_{config_stuff[param]}")

    # Initialize wandb run
    run = wandb.init(
        project=wandb_project_name,
        name=run_name,
        notes=model_descriptions[model_id],
        tags=tags,
        config=config_stuff
    )

    # TODO: call function above to get dict and another to initialize model
    special_tokens_dict = create_special_tokens_dict(model_id, model_args_dict)
    model, tokenizer = initialize_model_with_special_tokens(model_id, model_args_dict, model_hyperparameters, special_tokens_dict, model_descriptions)

    # TODO: call function for train_data_prepared
    train_data_prepared = encode_with_tokens_based_on_model_id(X_train, tokenizer, model_id, model_args_dict, model_hyperparameters)
    val_data_prepared = encode_with_tokens_based_on_model_id(X_val, tokenizer, model_id, model_args_dict, model_hyperparameters)

    # Log the model architecture
    wandb.watch(model.model, log='all')

    # Train the model
    model.train_model(train_data_prepared)

    # Evaluate the model on validation data
    result, model_outputs, wrong_predictions = model.eval_model(val_data_prepared)

    # Log evaluation metrics
    wandb.log(result)

    # Log an example input
    wandb.log({'example_input': train_data_prepared['text'].iloc[0]})

    # Log the model as an artifact with all necessary data for replication
    artifact = wandb.Artifact(
        name=f"model_{model_id}_{run.id}",
        type='model',
        description=model_descriptions[model_id],
        metadata={
            'model_id': model_id,
            'model_hyperparams': model_hyperparameters,
            'model_args': model_args_dict.to_dict(),
            'random_state': random_state,
        }
    )
    artifact.add_dir(model_args_dict.output_dir)
    run.log_artifact(artifact)

    # Finish wandb run
    run.finish()

    return model


Training Model 1: BERT*(tweet)
Training Model 2: BERT*({tweet <SEP> C1 <SEP> C2 ...})
Training Model 3: BERT*({tweet <FEAT> C1 <FEAT> C2 ...})


# Evaluation


In [22]:
def evaluate_model(X_test, model_id, model, tokenizer, model_hyperparameters, model_args_dict, wandb_project_name):
    """
    Evaluate the trained model on test data.

    Args:
        model: The trained model.
        X_test (DataFrame): Test data.
        hyperparameters (dict): Hyperparameters used during training.

    Returns:
        None
    """
    # Prepare test data
    test_data_prepared = encode_with_tokens_based_on_model_id(X_test, tokenizer, model_id, model_args_dict, model_hyperparameters)

    # Evaluate the model
    result, model_outputs, wrong_predictions = model.eval_model(test_data_prepared)

    # Log evaluation metrics to wandb
    wandb.log(result)

    true_labels = test_data_prepared['label']
    predictions = np.argmax(model_outputs, axis=1)

    # Compute confusion matrix
    cm = confusion_matrix(true_labels, predictions)

    # Plot confusion matrix
    plt.figure(figsize=(6,6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix')

    # Log confusion matrix
    wandb.log({"confusion_matrix": wandb.Image(plt)})
    plt.close()


Evaluating Model 1
Evaluating Model 2
Evaluating Model 3


# Sweep config and training function

In [1]:
# Define the hyperparameter space
sweep_config = {
    'method': 'grid',
    'metric': {
        'name': 'eval_loss',
        'goal': 'minimize'
    },
    # TODO
}

sweep_id = wandb.sweep(sweep_config, project=WANDB_PROJECT_NAME)

def sweep_train():
    # Initialize wandb
    wandb.init()
    # Get hyperparameters from wandb.config. I only want to sweep through hyper
    # TODO
    # Train the model
    # TODO
    # Evaluate the model on test data
    # TODO


# Start the sweep with wandb.agent
#TODO

NameError: name 'wandb' is not defined

# Conclusion & next steps
# Conclusion

- We have set up the notebook structure for training and evaluating multiple models.
- The data loading and feature functions are integrated as per your requirements.
- We have placeholders for training and evaluation functions where wandb integration will be added.
- Next steps include implementing the TODO sections and ensuring comprehensive wandb logging.

# Next Steps

- Implement the `prepare_data` function to handle data preparation for each model.
- Complete the `train_model` function with wandb integration, including logging hyperparameters, metrics, and artifacts.
- Implement the `evaluate_model` function to evaluate models on validation and test data.
- Fine-tune hyperparameters using wandb sweeps if desired.
- Analyze the results and prepare reports using wandb's reporting tools.
